In [17]:
#HOMEWORK
# 1. Clean up my messy notebook, create one with just the essential operations

# 2. Put the credentials in an environment variable commit to ADO

# 3. Learn how to create a personal notebook of fav visualizations I like in other publications like Guardian, the Economist,
#NY Times, research papers: this way figure out convention for visualization. For ex. avoid having categories on the axis.

# 4. look into bar charts. Try 2 quantitative axis. Avoid duplicating information with the size.Don't show the same thing
#twice!

# 5 Do CR by openees level, the dot size show the places where we are underperforming

# 6. Use the .query to see where the num_bookings are > 100 


In [19]:

# 2. Put the credentials in an environment variable commit to ADO

import pandas as pd
from sqlalchemy import create_engine
import os
clickh_user = os.getenv('CLICKHOUSE_USER')
clickh_pswd = os.getenv('CLICKHOUSE_PSWD')
clickh_name = os.getenv('CLICKHOUSE_NAME')
cnx = f"clickhouse://{clickh_user}:{clickh_pswd}@{clickh_name}"
engine = create_engine(cnx)
query = """ select DestCountry, DestCountryName, TID_2019, TID_2021, searches_plus_sherpa.* from (select DestCountry,
DestCountryName, TID_2019, TID_2021 FROM (select DestCountry, DestCountryName, count(1) TID_2019 from 
datascience.BookingDetails where Year_BookDate = 2019 and DestCountryName != 'United States' group by DestCountry,
DestCountryName, Year_BookDate) table_2019 JOIN (select DestCountry, DestCountryName, count(1) TID_2021 from datascience.BookingDetails
where Year_BookDate = 2021 and DestCountryName != 'United States'
group by DestCountry,DestCountryName, Year_BookDate) table_2021 ON table_2019.DestCountryName = table_2021.DestCountryName) historical
JOIN (select SearchDepartureToCountry, num_searches, sherpa.* FROM (select SearchDepartureToCountry, count(1) num_searches
from searches.searchlog s where SearchDate = today() group by SearchDepartureToCountry ) sa JOIN (select distinct DestCountry,
DestCountryName from datascience.BookingDetails) cm on sa.SearchDepartureToCountry = cm.DestCountry
join datascience.SherpaData sherpa ON sherpa.CountryName = cm.DestCountryName where sherpa.FullyVaccinated = 1) searches_plus_sherpa ON
historical.DestCountryName = searches_plus_sherpa.sherpa.CountryName
order by num_searches desc"""
search_sherpa_DF = pd.read_sql_query(query, engine)
search_sherpa_DF[0:10]

,DestCountry,DestCountryName,TID_2019,TID_2021,SearchDepartureToCountry,num_searches,sherpa.CountryCode,sherpa.CountryName,sherpa.FullyVaccinated,sherpa.TravelOpenness,sherpa.EntryRestrictions,sherpa.Covid19Test,sherpa.Quarantine,sherpa.Response,sherpa.UpdatedOn
0,IN,India,137027,45590,IN,7204410,IND,India,1,LEVEL_2,,,,"{""meta"":{""copyright"":""Sherpa"",""version"":""2.8.2...",2021-12-12 14:00:38.982000+00:00
1,CA,Canada,330572,111241,CA,6627915,CAN,Canada,1,LEVEL_2,,,,"{""meta"":{""copyright"":""Sherpa"",""version"":""2.8.2...",2021-12-12 14:00:38.982000+00:00
2,MX,Mexico,299667,188578,MX,5706275,MEX,Mexico,1,LEVEL_1,,,,"{""meta"":{""copyright"":""Sherpa"",""version"":""2.8.2...",2021-12-12 14:00:38.982000+00:00
3,GB,United Kingdom,89147,13895,GB,3450380,GBR,United Kingdom,1,LEVEL_3,,,,"{""meta"":{""copyright"":""Sherpa"",""version"":""2.8.2...",2021-12-12 14:00:38.982000+00:00
4,PH,Philippines,131742,20055,PH,2656779,PHL,Philippines,1,LEVEL_4,,,,"{""meta"":{""copyright"":""Sherpa"",""version"":""2.8.2...",2021-12-12 14:00:38.982000+00:00
5,AE,United Arab Emirates,23317,10636,AE,2589463,ARE,United Arab Emirates,1,LEVEL_2,,,,"{""meta"":{""copyright"":""Sherpa"",""version"":""2.8.2...",2021-12-12 14:00:38.982000+00:00
6,PK,Pakistan,18266,11444,PK,1957617,PAK,Pakistan,1,LEVEL_2,,,,"{""meta"":{""copyright"":""Sherpa"",""version"":""2.8.2...",2021-12-12 14:00:38.982000+00:00
7,IT,Italy,57524,8334,IT,1924469,ITA,Italy,1,LEVEL_2,,,,"{""meta"":{""copyright"":""Sherpa"",""version"":""2.8.2...",2021-12-12 14:00:38.982000+00:00
8,TR,Turkey,29911,19332,TR,1849838,TUR,Turkey,1,LEVEL_2,,,,"{""meta"":{""copyright"":""Sherpa"",""version"":""2.8.2...",2021-12-12 14:00:38.982000+00:00
9,FR,France,58945,9648,FR,1789220,FRA,France,1,LEVEL_2,,,,"{""meta"":{""copyright"":""Sherpa"",""version"":""2.8.2...",2021-12-12 14:00:38.982000+00:00


In [20]:
search_sherpa_DF.drop(['sherpa.EntryRestrictions', 'sherpa.Covid19Test', 'sherpa.Quarantine', 'sherpa.Response'], axis=1, inplace=True)

In [21]:
search_sherpa_DF

,DestCountry,DestCountryName,TID_2019,TID_2021,SearchDepartureToCountry,num_searches,sherpa.CountryCode,sherpa.CountryName,sherpa.FullyVaccinated,sherpa.TravelOpenness,sherpa.UpdatedOn
0,IN,India,137027,45590,IN,7204410,IND,India,1,LEVEL_2,2021-12-12 14:00:38.982000+00:00
1,CA,Canada,330572,111241,CA,6627915,CAN,Canada,1,LEVEL_2,2021-12-12 14:00:38.982000+00:00
2,MX,Mexico,299667,188578,MX,5706275,MEX,Mexico,1,LEVEL_1,2021-12-12 14:00:38.982000+00:00
3,GB,United Kingdom,89147,13895,GB,3450380,GBR,United Kingdom,1,LEVEL_3,2021-12-12 14:00:38.982000+00:00
4,PH,Philippines,131742,20055,PH,2656779,PHL,Philippines,1,LEVEL_4,2021-12-12 14:00:38.982000+00:00
...,...,...,...,...,...,...,...,...,...,...,...
178,MO,Macau,1112,14,MO,264,MAC,Macau,1,LEVEL_4,2021-12-12 14:00:38.982000+00:00
179,KI,Kiribati,67,1,KI,246,KIR,Kiribati,1,NO_INFORMATION,2021-12-12 14:00:38.982000+00:00
180,CK,Cook Islands,612,5,CK,167,COK,Cook Islands,1,LEVEL_4,2021-12-12 14:00:38.982000+00:00
181,KM,Comoros,37,22,KM,127,COM,Comoros,1,NO_INFORMATION,2021-12-12 14:00:38.982000+00:00


In [22]:
ratio_series = (search_sherpa_DF['TID_2019'])/(search_sherpa_DF['TID_2021'])

In [23]:
#Cast my series into a list to then be able to add the list as a column in my dataframe

ratio_list = ratio_series.values.tolist()

In [26]:
search_sherpa_DF['ratio_19/21'] = ratio_list

In [27]:
search_sherpa_DF

,DestCountry,DestCountryName,TID_2019,TID_2021,SearchDepartureToCountry,num_searches,sherpa.CountryCode,sherpa.CountryName,sherpa.FullyVaccinated,sherpa.TravelOpenness,sherpa.UpdatedOn,ratio_19/21
0,IN,India,137027,45590,IN,7204410,IND,India,1,LEVEL_2,2021-12-12 14:00:38.982000+00:00,3.005637
1,CA,Canada,330572,111241,CA,6627915,CAN,Canada,1,LEVEL_2,2021-12-12 14:00:38.982000+00:00,2.971674
2,MX,Mexico,299667,188578,MX,5706275,MEX,Mexico,1,LEVEL_1,2021-12-12 14:00:38.982000+00:00,1.589088
3,GB,United Kingdom,89147,13895,GB,3450380,GBR,United Kingdom,1,LEVEL_3,2021-12-12 14:00:38.982000+00:00,6.415761
4,PH,Philippines,131742,20055,PH,2656779,PHL,Philippines,1,LEVEL_4,2021-12-12 14:00:38.982000+00:00,6.569035
...,...,...,...,...,...,...,...,...,...,...,...,...
178,MO,Macau,1112,14,MO,264,MAC,Macau,1,LEVEL_4,2021-12-12 14:00:38.982000+00:00,79.428571
179,KI,Kiribati,67,1,KI,246,KIR,Kiribati,1,NO_INFORMATION,2021-12-12 14:00:38.982000+00:00,67.000000
180,CK,Cook Islands,612,5,CK,167,COK,Cook Islands,1,LEVEL_4,2021-12-12 14:00:38.982000+00:00,122.400000
181,KM,Comoros,37,22,KM,127,COM,Comoros,1,NO_INFORMATION,2021-12-12 14:00:38.982000+00:00,1.681818


In [28]:
#How to add a new column to an existing dataframe
#https://www.geeksforgeeks.org/adding-new-column-to-existing-dataframe-in-pandas/

#The column appears at the end of the dataframe. I want to move it closer to the TID_2019 and TID_2021
# Check out this video to https://sparkbyexamples.com/pandas/pandas-change-the-order-of-columns/
ratio_DF = search_sherpa_DF[['DestCountry', "DestCountryName", "TID_2019", "TID_2021", "ratio_19/21", "num_searches", "sherpa.TravelOpenness"]]

In [29]:
ratio_DF

,DestCountry,DestCountryName,TID_2019,TID_2021,ratio_19/21,num_searches,sherpa.TravelOpenness
0,IN,India,137027,45590,3.005637,7204410,LEVEL_2
1,CA,Canada,330572,111241,2.971674,6627915,LEVEL_2
2,MX,Mexico,299667,188578,1.589088,5706275,LEVEL_1
3,GB,United Kingdom,89147,13895,6.415761,3450380,LEVEL_3
4,PH,Philippines,131742,20055,6.569035,2656779,LEVEL_4
...,...,...,...,...,...,...,...
178,MO,Macau,1112,14,79.428571,264,LEVEL_4
179,KI,Kiribati,67,1,67.000000,246,NO_INFORMATION
180,CK,Cook Islands,612,5,122.400000,167,LEVEL_4
181,KM,Comoros,37,22,1.681818,127,NO_INFORMATION


In [30]:
import plotly.express as px

fig = px.scatter(ratio_DF, x='num_searches', y='TID_2021', color = 'sherpa.TravelOpenness', 
                 color_discrete_sequence= ["yellow", "green", "red", "orange", "blue"], size = 'ratio_19/21', 
                 hover_name="DestCountryName", trendline = "ols")
fig.show()

In [38]:
# 6. Use the .query to see where the num_bookings are > 10000 

search_sherpa_DF.query('TID_2019 > 10000')[0:20]

,DestCountry,DestCountryName,TID_2019,TID_2021,SearchDepartureToCountry,num_searches,sherpa.CountryCode,sherpa.CountryName,sherpa.FullyVaccinated,sherpa.TravelOpenness,sherpa.UpdatedOn,ratio_19/21
0,IN,India,137027,45590,IN,7204410,IND,India,1,LEVEL_2,2021-12-12 14:00:38.982000+00:00,3.005637
1,CA,Canada,330572,111241,CA,6627915,CAN,Canada,1,LEVEL_2,2021-12-12 14:00:38.982000+00:00,2.971674
2,MX,Mexico,299667,188578,MX,5706275,MEX,Mexico,1,LEVEL_1,2021-12-12 14:00:38.982000+00:00,1.589088
3,GB,United Kingdom,89147,13895,GB,3450380,GBR,United Kingdom,1,LEVEL_3,2021-12-12 14:00:38.982000+00:00,6.415761
4,PH,Philippines,131742,20055,PH,2656779,PHL,Philippines,1,LEVEL_4,2021-12-12 14:00:38.982000+00:00,6.569035
5,AE,United Arab Emirates,23317,10636,AE,2589463,ARE,United Arab Emirates,1,LEVEL_2,2021-12-12 14:00:38.982000+00:00,2.192272
6,PK,Pakistan,18266,11444,PK,1957617,PAK,Pakistan,1,LEVEL_2,2021-12-12 14:00:38.982000+00:00,1.596120
7,IT,Italy,57524,8334,IT,1924469,ITA,Italy,1,LEVEL_2,2021-12-12 14:00:38.982000+00:00,6.902328
8,TR,Turkey,29911,19332,TR,1849838,TUR,Turkey,1,LEVEL_2,2021-12-12 14:00:38.982000+00:00,1.547227
9,FR,France,58945,9648,FR,1789220,FRA,France,1,LEVEL_2,2021-12-12 14:00:38.982000+00:00,6.109556
